# Laboratorio: Recommendation con Python (senza librerie)

**Programmazione di Applicazioni Data Intensive**  
Laurea in Ingegneria e Scienze Informatiche  
DISI - Università di Bologna, Cesena

Proff. Gianluca Moro, Roberto Pasolini  
nome.cognome@unibo.it

## Introduzione a Jupyter/Colab

- Questo è un file Jupyter (`.ipynb`)
- Al suo interno si trovano celle di codice Python eseguibili
- Eseguendo una cella, il risultato che si ottiene è riportato sotto la cella stessa
  - stringhe stampate con `print` e/o risultato di un'espressione
- Le celle di codice possono essere modificate e rieseguite liberamente

- Questo è un esempio di cella di codice:

In [1]:
20 + 20 + 2

42

- Cliccare sulla cella e premere **Maiusc + Invio** per eseguirla: il risultato dell'espressione comparirà sotto
- Si può aggiungere una nuova cella di codice sotto a quella corrente
  - cliccando sul pulsante "+ Code" che appare tra le celle in Colab
  - cliccando sul pulsante "+" in alto in Jupyter

### Comandi principali da tastiera

- **Ctrl + Invio**: esegui cella corrente
- **Maiusc + Invio**: esegui cella corrente e seleziona la successiva
- **Esc**: termina modifica (senza eseguire)

I comandi sotto funzionano sulla cella selezionata solo se non se ne sta modificando il contenuto:
- **Invio**: modifica contenuto
- **A/B**: crea nuova cella sopra/sotto
- **Ctrl+M** seguito da **D**: elimina cella (due volte D in Jupyter)

### Autocompletamento e inline help

- Nelle celle di codice sono forniti suggerimenti sul completamento di un nome di variabile, funzione, ecc.
  - si provi ad es. a creare una cella di codice (tasto B), e digitare "so"
  - attivare l'autocompletamento con Ctrl+Invio su Colab o con Tab su Jupyter
  - l'autocompletamento suggerisce ad es. la funzione `sorted`
- Per ottenere informazioni in linea su una funzione o metodo `foo`, eseguire una cella con `foo?`
  - in alternativa, usare la funzione `help` di Python: `help(foo)`

In [2]:
len?

Signature: len(obj, /)
Docstring: Return the number of items in a container.
Type:      builtin_function_or_method


In [3]:
list.append?

Signature: list.append(self, object, /)
Docstring: Append object to the end of the list.
Type:      method_descriptor


## Recommendation: Prevedere le propensioni di acquisto (e molto altro...)

- Ogni azienda ha i dati storici di acquisto di ciascun cliente/utente
- Vogliamo **raccomandare/suggerire ai singoli utenti quali prodotti acquistare** 
  - idea: utenti con storie di acquisti simili faranno probabilmente acquisti simili anche in futuro 
  - metodo: proponiamo ad ogni utente gli acquisti fatti da altri utenti con storico più simile al proprio
- Vediamo come estrarre i suggerimenti sfruttando le **strutture dati standard di Python** e le operazioni che offrono
  - vedremo poi come farlo in modo più efficiente con operazioni tra matrici...
- Usiamo come esempio un set di dati ricavato da vendite su Amazon

## Scaricamento file dati

- Un archivio ZIP con i file necessari per l'esercitazione si trova all'URL https://git.io/fhxQh
- Eseguire la seguente cella di codice per
  - verificare se il file ZIP con i dati è già presente
  - scaricare il file ZIP se non è già presente
  - estrarre i file nella cartella corrente
- Si può vedere come la libreria standard di Python fornisca già funzioni per eseguire agevolmente queste operazioni

In [4]:
# importo moduli e funzioni necessarie dalla libreria standard
import os.path
from urllib.request import urlretrieve
from zipfile import ZipFile

# se il file "purchases_data.zip" non esiste
if not os.path.exists("purchases_data.zip"):
    # scarica il file dall'URL indicato
    urlretrieve("https://git.io/fhxQh", "purchases_data.zip")
    # apri il file zip ed estrai tutto il contenuto nella directory corrente
    with ZipFile("purchases_data.zip") as f:
        f.extractall()

## Caricamento nomi utenti

- Il file `users.csv` contiene un elenco degli utenti coinvolti nell'analisi
  - sono stati selezionati gli utenti con almeno 30 acquisti nello storico
- È un file CSV (_Comma Separated Values_) contenente una riga per ogni utente nel formato `IdUtente;Nome`
- Possiamo usare il modulo `csv` della libreria standard di Python per leggere tali file in modo semplice
- Importiamo il modulo per caricarlo in memoria

In [5]:
import csv

- Creiamo un dizionario `users` e inseriamo in esso i dati letti dal file
  - vogliamo inserire un elemento per ogni utente, la cui chiave sia l'ID e il cui valore sia il nome

In [6]:
users = {}

- Per **aprire il file** utilizziamo la funzione `open` specificando il nome del file
  - il file è aperto in lettura (`"r"`) e modalità testo

In [7]:
f = open("users.csv", "r")

- L'oggetto `f` può essere iterato per ottenere le righe del file una alla volta
- Usiamo però `csv.reader` su `f` per scomporre ogni riga in una **tupla di valori**
  - specifichiamo che i valori sono separati da ";"

In [8]:
reader = csv.reader(f, delimiter=";")

- Col costrutto `for` iteriamo le righe del file
  - iterando su tuple di 2 elementi ciascuna, l'_unpacking_ ci permette di scomporre ciascuna tupla in due variabili
- Per ciascuna, inseriamo nel dizionario la coppia ID-nome
  - nel far ciò, convertiamo l'ID da stringa a numero `int`

In [9]:
for uid, name in reader:
    users[int(uid)] = name

- Una volta finito di leggere il file, lo chiudiamo

In [10]:
f.close()

### Usare il costrutto with

- Quando si lavora con un file, è consigliabile gestirne apertura e chiusura col costrutto `with`
  - il file viene chiuso automaticamente all'uscita dell blocco `with`, anche in caso d'errore
- Come esempio, ripetiamo la creazione e popolazione della struttura dati `users`

In [11]:
users = {}
with open("users.csv", "r") as f:
    reader = csv.reader(f, delimiter=";")
    for uid, name in reader:
        users[int(uid)] = name

### Usare le comprehension

- Abbiamo creato una collezione (un dizionario) rielaborando gli elementi (righe) di un iterabile (file)
- Operazioni come questa si possono esprimere in Python in modo più compatto con le _comprehension_
  - `f(a) for a in X` indica "da ciascun elemento `a` estratto da un iterabile `X` calcola un'espressione `f(a)`"
- Per generare un dizionario scriviamo `{k: v for a in X}`
  - per ciascun elemento `a` in `X` otteniamo una coppia con chiave `k` e valore `v`
- Un modo equivalente per ottenere il dizionario `users` sopra è quindi questo:
  - (scriviamo la comprehension su più righe per evidenziarne le parti, possiamo farlo perché dentro parentesi)

In [12]:
with open("users.csv", "r") as f:
    users = {
        int(uid): name
        for uid, name
        in csv.reader(f, delimiter=";")
    }

- Dal dizionario possiamo reperire il nome di un qualsiasi utente dato il suo ID

In [13]:
users[84]

'malachix'

- Tramite la funzione `len` possiamo contare il numero totale di utenti

In [14]:
len(users)

178

### Esercizio 1: Caricamento nomi prodotti

- Il file `items.csv` contiene i prodotti distinti acquistati dagli utenti sopra
- Il formato del file è analogo a quello sopra, con righe `IdProdotto;Nome`
- Ne salviamo il contenuto in un dizionario `items`, ottenuto come fatto sopra con `users`

- **(1a)** Costruire un dizionario `items` in modo analogo a `users`
  - anche quì assicurarsi di convertire gli ID prodotti in numeri `int`
- **(1b)** Ottenere il numero totale di prodotti
- **(1c)** Ottenere il nome del prodotto con ID 2669

_(selezionare questa cella di testo e premere B per inserire una nuova cella di codice sotto in cui svolgere gli esercizi)_

In [16]:
with open("items.csv", "r") as f:
    items = {int(iid): name for iid, name in csv.reader(f, delimiter=";")}

In [17]:
len(items)

3384

In [18]:
items[2669]

'Independence Day [VHS]'

## Caricamento dati acquisti

- Il file CSV `purchases-2000.csv` contiene i dati sugli acquisti effettuati dagli utenti analizzati fino alla fine del 2000
- Per ciascun acquisto registrato, il file contiene una riga `IdUtente;IdProdotto`
- Usiamo le funzioni viste sopra per leggere il file, creando stavolta un insieme (`set`) di tuple `(uid, iid)`
  - i `set` sono collezioni di oggetti senza un ordine definito (seguono il concetto matematico di insieme)
  - i `set` possono contenere solo oggetti _immutabili_, come ad es. numeri, stringhe e tuple di oggetti a loro volta immutabili

In [19]:
with open("purchases-2000.csv", "r") as f:
    purchases = set(
        (int(uid), int(iid))
        for uid, iid
        in csv.reader(f, delimiter=";")
    )

### Esercizio 2: Analisi acquisti

- **(2a)** Ottenere il numero di tuple caricate dal file
- **(2b)** Ottenere il numero medio di acquisti effettuati da ogni utente

In [21]:
len(purchases)

9683

In [22]:
len(purchases) / len(users)

54.39887640449438

## Raggruppare gli acquisti per utente

- Per lavorare agevolmente con questi dati, estraiamo un dizionario `purchases_by_user` che associ ad ogni ID utente l'insieme di ID dei prodotti che ha acquistato
  - usiamo un ciclo `for` per scorrere tutte le coppie utente U - prodotto P
  - per ogni coppia, se all'utente U non è associato un insieme di acquisti, ne associamo uno vuoto
  - quindi aggiungiamo P all'insieme acquisti di U

In [23]:
purchases_by_user = {}
for uid, iid in purchases:
    if uid not in purchases_by_user:
        purchases_by_user[uid] = set()
    purchases_by_user[uid].add(iid)

- Possiamo abbreviare il codice sfruttando il metodo `setdefault` dei dizionari
  - `d.setdefault(k, v)` restituisce `d[k]` impostandolo prima a `v` se non esistente

In [24]:
purchases_by_user = {}
for uid, iid in purchases:
    purchases_by_user.setdefault(uid, set()).add(iid)

- Possiamo verificare che ogni utente abbia effettivamente almeno 30 acquisti
  - col metodo `values` iteriamo gli insiemi di oggetti acquistati (i soli valori, non le chiavi) nel dizionario `purchases_by_user`
  - con `len` estraiamo il numero di elementi di ciascuno
  - con `min` estraiamo il più piccolo di essi e verifichiamo che sia 30

In [25]:
min(len(itemset) for itemset in purchases_by_user.values())

30

- In alternativa alla comprehension possiamo usare la funzione `map(f, i)`, che applica una funzione `f` (quì `len`) a tutti gli elementi di un iterabile `i`

In [26]:
min(map(len, purchases_by_user.values()))

30

### Esercizio 3: Raggruppare gli acquisti per prodotto

- **(3a)** Costruire in modo simile a `purchases_by_user` un dizionario `purchases_by_item` che associ ad ogni ID di un _prodotto_ l'insieme di ID degli _utenti_ che l'hanno acquistato
- **(3b)** Ottenere il numero di acquisti del prodotto più venduto e di quello meno venduto

In [28]:
purchases_by_item = {}
for uid, iid in purchases:
    purchases_by_item.setdefault(iid, set()).add(uid)

In [29]:
max(map(len, purchases_by_item.values()))

50

In [30]:
min(map(len, purchases_by_item.values()))

1

## Similarità tra utenti

- Vogliamo suggerire prodotti agli utenti in base a **cos'hanno acquistato utenti simili**
- Come determinare quanto due utenti siano "simili"?
- Possiamo contare **quanti sono i prodotti che entrambi hanno acquistato**
- Per ottenere i prodotti acquistati da entrambi due utenti, possiamo calcolare **l'intersezione** degli insiemi dei prodotti acquistati
- Sugli insiemi si può usare l'operatore `&` (AND) per calcolare l'intersezione
- Ad esempio, gli ID dei prodotti acquistati sia dall'utente con ID 84 che da quello con ID 7661 sono:

In [31]:
purchases_by_user[84] & purchases_by_user[7661]

{5162, 43911, 43921, 100267}

### Esercizio 4: Funzione per calcolo similarità

- Creare una funzione `user_similarity` che, dati due ID utente `uid1` e `uid2`, restituisca il numero di prodotti nell'intersezione dei loro acquisti
  - completare la definizione abbozzata sotto

In [34]:
def user_similarity(uid1, uid2):
    """Count products purchased by both given users."""
    return len(purchases_by_user[uid1] & purchases_by_user[uid2])

- Eseguire la cella sotto per verificare la funzione sulla base dell'esempio sopra
  - se la condizione data è soddisfatta non succede nulla, altrimenti `assert` genera un errore

In [35]:
assert user_similarity(84, 7661) == 4

## Calcolo di tutte le similarità

- Usiamo questa funzione per creare un dizionario `user_similiarities` che, ad ogni tupla con due ID utente, associa la loro similarità
  - scorriamo tutti gli ID utente attraverso due cicli (`for`) innestati, eliminando le coppie di ID uguali
  - _(per semplicità, lasciamo che la similarità di ogni coppia sia calcolata due volte)_

In [36]:
user_similarities = {
    (i, j): user_similarity(i, j)
    for i in users.keys()
    for j in users.keys()
    if i != j
}

- Ad esempio, riprendendo l'esempio sopra, la similarità tra gli utenti 84 e 7661 deve essere 4

In [37]:
user_similarities[(84, 7661)]

4

## Stimare il potenziale interesse nei prodotti

- Vogliamo stimare **quanto ciascun utente sia potenzialmente interessato** in ciascun prodotto non ancora acquistato
- Possiamo stimarlo in base a quanto il prodotto **sia stato acquistato da utenti simili**
- Associamo per ogni utente U e prodotto P un _punteggio d'interesse_ pari alla **somma delle similarità degli altri utenti** che hanno acquistato P

### Esercizio 5: Funzione per la stima dell'interesse

- Creare una funzione `interest` che calcoli tale punteggio per un utente `uid` e un prodotto `iid` dati
  - assicurarsi di iterare gli utenti che hanno acquistato `iid` escludendo `uid`
  - usare la funzione `sum(...)` per sommare i valori ricavati da una comprehension

In [40]:
def interest(uid, iid):
    """Estimate the interest of given user for given product."""
    return sum(
        user_similarities[(uid, ouid)]
        for ouid in purchases_by_item[iid]
        if uid != ouid
    )

### Calcolo di tutti i punteggi d'interesse

- Raccogliamo tutti i punteggi in un dizionario che associa ad ogni utente U un dizionario di punteggi d'interesse
  - ciascuno associa a sua volta a ciascun prodotto il punteggio d'interesse
  - sono però esclusi i prodotti già acquistati da U

In [41]:
interests_by_user = {
    uid: {
        iid: interest(uid, iid)
        for iid in items.keys()
        if iid not in purchases_by_user[uid]
    } for uid in users.keys()
}

- Ad esempio il punteggio d'interesse dell'utente 84 verso l'oggetto 2669 è

In [42]:
interests_by_user[84][2669]

57

### Esercizio 6: Estrazione punteggio massimo

- Estrarre il valore del punteggio d'interesse massimo tra quelli calcolati

In [44]:
max(
    interest_value
    for user_interests in interests_by_user.values()
    for interest_value in user_interests.values()
)

462

## Ottenere _N_ suggerimenti di acquisto per ogni utente

- Da migliaia di prodotti nel catalogo, vogliamo suggerirne **un numero limitato ad ogni utente** massimizzando la probabilità di acquisto
- Fissiamo un numero _N_ di prodotti da suggerire...

In [45]:
N = 20

- ...vogliamo selezionare per ogni utente gli **_N_ prodotti con "potenziale interesse" maggiore**

- Estraiamo ad esempio _N_ prodotti suggeriti per l'utente con ID 84
- Da `interests_by_user` estraiamo delle tuple `(IdOggetto, Punteggio)`
  - usiamo il metodo `items` per estrarre tutte le coppie chiave-valore di un dizionario in forma di tuple

In [46]:
interests_of_user_84 = interests_by_user[84].items()

- Usiamo la funzione `sorted` per ottenere una lista con gli elementi in ordine
  - col parametro `key` specifichiamo un criterio d'ordinamento
    - usiamo una funzione lambda che estragga da ogni tupla `x` il secondo elemento `x[1]`, cioè il punteggio
  - normalmente l'ordinamento è in ordine crescente, impostiamolo in ordine decrescente con `reverse=True`

In [47]:
sorted_interests_of_user_84 = \
    sorted(interests_of_user_84,
           key=lambda x: x[1],
           reverse=True)

- Abbiamo così una lista di tuple `(IdOggetto, Punteggio)` ordinate da quelle con punteggio maggiore
  - vediamo ad esempio le prime 5

In [48]:
sorted_interests_of_user_84[:5]

[(57372, 127), (59817, 97), (44030, 96), (57190, 86), (7989, 73)]

### Esercizio 7: Ottenere i suggerimenti per un utente

- **(7a)** Estrarre da quest'ultima lista un insieme degli ID degli _N_ prodotti da suggerire all'utente
  - estrarre solo gli ID dei prodotti, scartando i punteggi
- **(7b)** Definire una funzione `suggest` che, dato un ID utente `uid` arbitrario, restituisca un insieme simile di ID di _N_ prodotti seguendo la procedura descritta

In [51]:
suggestions_for_user_84 = set(iid for iid, score in sorted_interests_of_user_84[:N])

In [53]:
def suggest(uid):
    """Recommend N products to given user."""
    interests = interests_by_user[uid].items()
    sorted_interests = sorted(interests, key=lambda x: x[1], reverse=True)
    return set(iid for iid, score in sorted_interests[:N])

- Applichiamo la funzione così definita a tutti gli utenti

In [54]:
suggestions_by_user = {uid: suggest(uid) for uid in users.keys()}

- Abbiamo così per ciascun utente un set di _N_ prodotti non precedentemente acquistati da suggerire

In [55]:
print(suggestions_by_user[84])

{44037, 60041, 96025, 43290, 57372, 5288, 59817, 7985, 7989, 43586, 60230, 96454, 96456, 95843, 57190, 2669, 101103, 56561, 95480, 44030}


- Ad esempio, per l'utente 84, stampiamo i titoli dei film che ha acquistato...
  - sostituire `"; "` con `"\n"` (interruzione di riga) per visualizzare i titoli uno sotto l'altro

In [56]:
print("; ".join(items[iid] for iid in purchases_by_user[84]))

Eyes Wide Shut [VHS]; The Lion King [VHS]; The Matrix; Batman & Robin [VHS]; Blade [VHS]; First Knight [VHS]; Omen 3: The Final Conflict [VHS]; Summer of Sam [VHS]; The World Is Not Enough [VHS]; The Chinese Connection [VHS]; Enter the Dragon [VHS]; Star Wars - Episode I, The Phantom Menace [VHS]; Fists of Fury [VHS]; Lost World: Jurassic Park [VHS]; Alien [VHS]; Batman Returns (1992); Beloved; Reservoir Dogs [VHS]; Total Recall; Touch of Evil [VHS]; Batman Forever; Inspector Gadget; Enemy of the State; Blade Runner (The Director's Cut); Tomorrow Never Dies (Limited Edition Gift Pack) [VHS]; Aliens [VHS]; A Bug's Life; Return of the Dragon [VHS]; Batman (1989); The Exorcist; Lost in Space [VHS]; GoldenEye (Special Edition); Excalibur [VHS]; Alien Resurrection [VHS]; Alien 3 [VHS]; Godzilla [VHS]; Game of Death [VHS]; Jurassic Park (Widescreen Edition) [VHS]


- ...e i titoli dei film suggeriti

In [57]:
print("; ".join(items[iid] for iid in suggestions_by_user[84]))

Double Jeopardy; Deep Blue Sea; American Pie - Rated Edition (Special Edition) [VHS]; The Green Mile [VHS]; The Sixth Sense [VHS]; Being John Malkovich; Saving Private Ryan [VHS]; The Insider; Fight Club; Galaxy Quest [VHS]; Three Kings; The Talented Mr. Ripley; Titanic [VHS]; Jaws [VHS]; Curse of the Blair Witch [VHS]; Independence Day [VHS]; Abyss [VHS]; Armageddon [VHS]; Dogma [VHS]; Sleepy Hollow


## Accuratezza dei suggerimenti di acquisto

- Come valutare se i suggerimenti ottenuti in questo modo siano azzeccati?
- Una possibilità consiste nel verificare **se gli oggetti suggeriti siano stati effettivamente acquistati** in un successivo momento
- Nel file `purchases-2014.csv` è fornita una seconda lista di acquisti aggiornata, che include anche quelli successivi al 2000
  - utenti e prodotti sono limitati a quelli già caricati in `users` e `items`
- Possiamo quindi confrontare i prodotti suggeriti con questa nuova matrice
- Carichiamo l'insieme di tuple da questo file come abbiamo fatto per il precedente

In [58]:
with open("purchases-2014.csv", "r") as f:
    purchases_updated = set(
        (int(uid), int(iid))
        for uid, iid
        in csv.reader(f, delimiter=";")
    )

### Selezionare solo i nuovi acquisti

- Il nuovo file riporta **tutti** gli acquisti, compresi quelli già indicati nel file precedente
- Vogliamo un insieme dei soli acquisti successivi all'analisi svolta sopra
- Possiamo ottenerlo calcolando la differenza tra gli insiemi di acquisti, tramite l'operatore `-`

In [59]:
new_purchases = purchases_updated - purchases

### Esercizio 8: Caricamento e analisi dati nuovi acquisti

- **(8a)** Costruire un dizionario `new_purchases_by_user` simile a `purchases_by_user` creato in precedenza, con i nuovi acquisti raggruppati per utente
- **(8b)** Individuare il numero di nuovi acquisti massimo e quello medio per ogni utente

In [61]:
new_purchases_by_user = {}
for uid, iid in new_purchases:
    new_purchases_by_user.setdefault(uid, set()).add(iid)

In [62]:
max(map(len, new_purchases_by_user.values()))

431

In [63]:
sum(map(len, new_purchases_by_user.values())) / len(users)

32.98314606741573

### Quali nuovi acquisti sono stati suggeriti?

- Abbiamo ora i dizionari
  - `suggestions_by_user` con gli acquisti _suggeriti_
  - `new_purchases_by_user` con i nuovi acquisti _effettivi_
- Da questi possiamo individuare quali sono i suggerimenti **validi**, quelli a cui dopo l'analisi è corrisposto un acquisto
- Consideriamo un utente _soddisfatto_ se ha ricevuto **almeno un suggerimento valido**
- Individuiamo l'insieme degli utenti soddisfatti individuando quelli dove l'intersezione tra suggerimenti e nuovi acquisti non è vuota
  - usando un insieme (o altra collezione) in `if`, otteniamo `True` se e solo se l'insieme non è vuoto
  - dato che alcuni utenti non sono presenti come chiavi in `new_purchases_by_user`, usiamo il metodo `d.get(k, v)` per restituire un valore default `v` se `d[k]` non esiste

In [64]:
satisfied_users = {uid for uid in users.keys()
                   if suggestions_by_user[uid] & new_purchases_by_user.get(uid, set())}

- Quanti sono gli utenti soddisfatti?

In [65]:
len(satisfied_users)

62

- Quanti sono come percentuale rispetto al totale degli utenti analizzati?

In [66]:
len(satisfied_users) / len(users)

0.34831460674157305

- Abbiamo quindi suggerito **almeno un prodotto valido** per circa **un terzo degli utenti**

## Sviluppi successivi

- Abbiamo quì visto come usare le strutture dati e le funzioni standard di Python per un compito pratico
- Nel prossimo laboratorio vedremo come ottenere lo stesso risultato tramite **operazioni tra matrici e algebra lineare**

## Esercizio avanzato: Confronto con una selezione casuale di prodotti

_(si propone come esercizio da svolgere opzionalmente dopo la lezione)_

- Per valutare quanto il risultato ottenuto sia buono, possiamo misurare cosa otterremmo **suggerendo _N_ prodotti a caso** a ciascun utente
- Con questa procedura, si verifica che si otterrebbe una percentuale di clienti soddisfatti intorno al **13%**, contro il **35%** ottenuto sopra
- Per generare numeri casuali, usiamo il modulo `random` di Python

In [67]:
import random

- Per ottenere risultati riproducibili, impostiamo un valore fisso come seed

In [68]:
random.seed(1234567)

- **1)** creare una funzione `suggest_random` che, dato un ID utente, restituisca una lista di _N_ ID prodotti casuali tra quelli che non risultano da lui acquistati nel dataset del 2000
  - creare una lista con gli ID dei prodotti non acquistati da `uid`, ovvero le chiavi di `interests_by_user[uid]`
  - usare la funzione `sample(lista, k)` del modulo `random` per selezionare una lista di k elementi casuali dalla lista
  - convertire tale lista in un `set` per compatibilità con i passaggi successivi

In [71]:
def suggest_random(uid):
    unpurchased_iids = list(interests_by_user[uid].keys())
    suggested_iids = random.sample(unpurchased_iids, N)
    return set(suggested_iids)

- **2)** usare la funzione per creare un dizionario `random_suggestions_by_user` che associ ad ogni utente i suoi suggerimenti casuali
  - come riferimento si usi la creazione del dizionario `suggestions_by_user` sopra

In [72]:
random_suggestions_by_user = {uid: suggest_random(uid) for uid in users.keys()}

- **3)** creare un insieme `randomly_satisfied_users` con gli ID degli utenti per cui almeno un prodotto tra i suggerimenti casuali è stato acquistato in seguito
  - usare `satisfied_users` come riferimento

In [73]:
randomly_satisfied_users = {uid for uid in users.keys()
                            if random_suggestions_by_user[uid] & new_purchases_by_user.get(uid, set())}

- **4)** calcolare la percentuale di utenti in `randomly_satisfied_users` rispetto al totale

In [74]:
len(randomly_satisfied_users) / len(users)

0.11797752808988764

- **Extra.** Questa percentuale può cambiare variando il valore `seed` in alto. Rieseguire i calcoli sopra con 2-3 seed differenti, quindi calcolare la percentuale media su 1.000 prove con seed diversi.

In [75]:
results = []
for n in range(1000):
    random.seed(n)
    random_suggestions_by_user = {uid: suggest_random(uid) for uid in users.keys()}
    randomly_satisfied_users = {uid for uid in users.keys()
                                if random_suggestions_by_user[uid] & new_purchases_by_user.get(uid, set())}
    results.append(len(randomly_satisfied_users) / len(users))

sum(results) / len(results)

0.13008988764044957